In [3]:
import numpy as np 
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
plt.style.use('ggplot')
from tqdm import tqdm
import re
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import gensim
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.colors as mcolors
from sklearn.manifold import TSNE
from gensim.models import word2vec
import nltk
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
import json

### Loading of dataset

In [4]:
# Loading of dataset
data = pd.read_excel('Amazon.xlsx')

C:\Users\hp\AppData\Local\Temp\ipykernel_12408\3685901613.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  data = pd.read_excel('Amazon.xlsx')


In [5]:
data.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,1797882.0,R3I2DHQBR577SS,B001ANOOOE,2102612.0,The Naked Bee Vitmin C Moisturizing Sunscreen ...,Beauty,5.0,0.0,0.0,N,Y,Five Stars,"Love this, excellent sun block!!",2015-08-31
1,US,18381298.0,R1QNE9NQFJC2Y4,B0016J22EQ,106393691.0,"Alba Botanica Sunless Tanning Lotion, 4 Ounce",Beauty,5.0,0.0,0.0,N,Y,Thank you Alba Bontanica!,The great thing about this cream is that it do...,2015-08-31
2,US,19242472.0,R3LIDG2Q4LJBAO,B00HU6UQAG,375449471.0,"Elysee Infusion Skin Therapy Elixir, 2oz.",Beauty,5.0,0.0,0.0,N,Y,Five Stars,"Great Product, I'm 65 years old and this is al...",2015-08-31
3,US,19551372.0,R3KSZHPAEVPEAL,B002HWS7RM,255651889.0,"Diane D722 Color, Perm And Conditioner Process...",Beauty,5.0,0.0,0.0,N,Y,GOOD DEAL!,I use them as shower caps & conditioning caps....,2015-08-31
4,US,14802407.0,RAI2OIG50KZ43,B00SM99KWU,116158747.0,Biore UV Aqua Rich Watery Essence SPF50+/PA+++...,Beauty,5.0,0.0,0.0,N,Y,this soaks in quick and provides a nice base f...,This is my go-to daily sunblock. It leaves no ...,2015-08-31


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30014 entries, 0 to 30013
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   marketplace        30009 non-null  object        
 1   customer_id        30009 non-null  float64       
 2   review_id          30009 non-null  object        
 3   product_id         30009 non-null  object        
 4   product_parent     30009 non-null  float64       
 5   product_title      30014 non-null  object        
 6   product_category   30009 non-null  object        
 7   star_rating        30009 non-null  float64       
 8   helpful_votes      30009 non-null  float64       
 9   total_votes        30009 non-null  float64       
 10  vine               30009 non-null  object        
 11  verified_purchase  30009 non-null  object        
 12  review_headline    30007 non-null  object        
 13  review_body        30002 non-null  object        
 14  review

### Preprocessing of dataset

In [7]:
# Filter out reviews with zero helpful votes
df = data[data['helpful_votes'] > 0]

# Filter out rows with nan values
df = df.dropna()

# Reset index
df = df.reset_index(drop=True)

# convert float to int
df['star_rating'] = df['star_rating'].astype(int)
df['helpful_votes'] = df['helpful_votes'].astype(int)

In [8]:
df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,24655453.0,R2A30ALEGLMCGN,B00SAQ9DZY,292127037.0,"12 New, High Quality, Amber 2 ml (5/8 Dram) Gl...",Beauty,4,1,1.0,N,Y,Good Product,"These are great for small mixtures for EO's, e...",2015-08-31
1,US,11257536.0,R6CE3SOIUJGP4,B00PYL8MAA,390030149.0,Proganix Agave Nectar Plus Silica Curling Crea...,Beauty,4,1,1.0,N,N,Love this cream !,Wish I had discovered this years ago ! Leaves ...,2015-08-31
2,US,23620123.0,R3I4BQ6654MQNA,B00FWXBLHG,464001209.0,Vintage Lil' Sponge Holder,Beauty,5,1,1.0,N,Y,"Great product, fast delivery",I'm in love with this! It's so unique and fits...,2015-08-31
3,US,25564245.0,R3LUPG356F1D40,B003KL8CB0,328680790.0,Jenna Jameson Heartbreaker Perfume parent,Beauty,5,3,3.0,N,N,Does not smell cheap!,I was given this product in exchange for a rev...,2015-08-31
4,US,22393078.0,R2QRUE9REK8OUC,B00461F4PA,608719013.0,"Baby Foot Exfoliant Foot Peel, Lavender Scente...",Beauty,5,18,23.0,N,Y,Better Than Any Pedi,"First off, I'll say I'm skeptical. I've tried ...",2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7504,US,1856513.0,R1NP7JNUKOQYJR,B0002ARF4C,656652514.0,Lafeber's Premium Daily Diet Pellets for Cocka...,Pet Products,5,2,2.0,N,Y,Fantastic product,Fantastic product. I have been using this bran...,2015-08-30
7505,US,49358505.0,R1SOYNMVFQY125,B0009YUFMK,238149518.0,Pet Supply Imports Herm Sprenger Chrome Plated...,Pet Products,1,2,15.0,N,N,If you are this lazy hire someone else to trai...,These are for people who have no idea how to t...,2015-08-30
7506,US,43339631.0,R3AS69IN0CCBEH,B008CO5IUI,793758338.0,"Samsung DW7933LRABB 24"" Black Full Console Dis...",Major Appliances,1,3,3.0,N,N,The only good thing was it was quiet,This is my first product review. My advice is...,2015-07-22
7507,US,13150882.0,R2M3HM87H61ETP,B007A1XG5I,730919193.0,New Chapter Every Woman's One Daily 40 Plus Bo...,Health & Personal Care,5,3,4.0,N,Y,Amazon Prime is the best and most cost effecti...,I have been taking these vitamins on and off f...,2015-08-31


In [9]:
review=pd.DataFrame(df.groupby('star_rating').size().sort_values(ascending=False).rename('No of Users').reset_index())
review.head()

,star_rating,No of Users
0,5,4125
1,1,1345
2,4,987
3,3,574
4,2,478


In [10]:
# subset of data we will be working with to train our model
data = df[['review_body','star_rating','helpful_votes']]
data

,review_body,star_rating,helpful_votes
0,"These are great for small mixtures for EO's, e...",4,1
1,Wish I had discovered this years ago ! Leaves ...,4,1
2,I'm in love with this! It's so unique and fits...,5,1
3,I was given this product in exchange for a rev...,5,3
4,"First off, I'll say I'm skeptical. I've tried ...",5,18
...,...,...,...
7504,Fantastic product. I have been using this bran...,5,2
7505,These are for people who have no idea how to t...,1,2
7506,This is my first product review. My advice is...,1,3
7507,I have been taking these vitamins on and off f...,5,3


In [11]:
dummies = pd.get_dummies(data['star_rating'])
dummies.head()

,1,2,3,4,5
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1


### Preprocessing of text in review_body

In [12]:
## Convert to lowercase
data['review_body'] = data['review_body'].str.lower()
data.head()

C:\Users\hp\AppData\Local\Temp\ipykernel_12408\3765309073.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review_body'] = data['review_body'].str.lower()


,review_body,star_rating,helpful_votes
0,"these are great for small mixtures for eo's, e...",4,1
1,wish i had discovered this years ago ! leaves ...,4,1
2,i'm in love with this! it's so unique and fits...,5,1
3,i was given this product in exchange for a rev...,5,3
4,"first off, i'll say i'm skeptical. i've tried ...",5,18


In [13]:
## Remove punctuations

import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

data['review_body'] = data['review_body'].apply(lambda text: remove_punctuation(text))
data.head()

C:\Users\hp\AppData\Local\Temp\ipykernel_12408\1904549873.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review_body'] = data['review_body'].apply(lambda text: remove_punctuation(text))


,review_body,star_rating,helpful_votes
0,these are great for small mixtures for eos esp...,4,1
1,wish i had discovered this years ago leaves m...,4,1
2,im in love with this its so unique and fits wi...,5,1
3,i was given this product in exchange for a rev...,5,3
4,first off ill say im skeptical ive tried tons ...,5,18


In [14]:
## Removal of Stopwords
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

data['review_body'] = data['review_body'].apply(lambda text: remove_stopwords(text))
data.head()

C:\Users\hp\AppData\Local\Temp\ipykernel_12408\1673002800.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review_body'] = data['review_body'].apply(lambda text: remove_stopwords(text))


,review_body,star_rating,helpful_votes
0,great small mixtures eos especially traveling ...,4,1
1,wish discovered years ago leaves curles super ...,4,1
2,im love unique fits decor beautifully couldnt ...,5,1
3,given product exchange review say wow love per...,5,3
4,first ill say im skeptical ive tried tons prod...,5,18


In [15]:
## Removal of urls
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

data['review_body'] = data['review_body'].apply(lambda text: remove_urls(text))
data.head()

C:\Users\hp\AppData\Local\Temp\ipykernel_12408\541465091.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review_body'] = data['review_body'].apply(lambda text: remove_urls(text))


,review_body,star_rating,helpful_votes
0,great small mixtures eos especially traveling ...,4,1
1,wish discovered years ago leaves curles super ...,4,1
2,im love unique fits decor beautifully couldnt ...,5,1
3,given product exchange review say wow love per...,5,3
4,first ill say im skeptical ive tried tons prod...,5,18


In [16]:
x_train, x_test, y_train, y_test = train_test_split(
    data['review_body'], 
    dummies, 
    test_size=0.3, random_state=19
)

In [17]:
def build_matrix(word_index, path):
    def get_coefs(word, *arr):
        return word, np.asarray(arr, dtype='float32')

    def load_embeddings(path):
        with open(path, encoding='utf-8') as f:
            embedding_index = {}
            
            for line in tqdm(f):
                word, arr = get_coefs(*line.strip().split(' '))    
                if word in word_index:
                    embedding_index[word] = arr
            
        return embedding_index

    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    
    for word, i in tqdm(word_index.items()):
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            pass
    return embedding_matrix

In [18]:
def build_model(embedding_matrix):
    words = Input(shape=(None,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(LSTM(256, return_sequences=True))(x)
 
    hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
    hidden = Dense(512, activation='relu')(hidden)
    
    result = Dense(5, activation='softmax')(hidden)
    
    model = Model(inputs=words, outputs=result)
    model.compile(
        loss='categorical_crossentropy', 
        optimizer='adam',
        metrics=['accuracy']
    )

    return model

In [19]:
%%time
CHARS_TO_REMOVE = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'
tokenizer = text.Tokenizer(filters=CHARS_TO_REMOVE)
tokenizer.fit_on_texts(list(x_train) + list(x_test))

CPU times: total: 375 ms
Wall time: 384 ms


In [20]:
# Requires crawl-300d-2M.vec from https://fasttext.cc/docs/en/english-vectors.html
embedding_matrix = build_matrix(tokenizer.word_index, './crawl-300d-2M.vec')

1999996it [03:16, 10156.56it/s]
100%|██████████| 23148/23148 [00:00<00:00, 446354.76it/s]


In [21]:
x_train

966     ophthalmologist actually recommended makeup re...
6162    great item good price opened started cleaning ...
7036         cats love prime member get free day delivery
6601    seat cover appears well made absolutely love a...
382     second foot file tried awful proclaim flexible...
                              ...                        
5032                                                waste
1378         love microwave gives space kitchen con price
757     product great previously rechargeable model sh...
2670     great look fit finish would definitely recommend
5725    bought gag gift actually great stretchy fits s...
Name: review_body, Length: 5256, dtype: object

In [22]:
maxlen=512
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
x_train = pad_sequences(x_train, maxlen=512)
x_test = pad_sequences(x_test, maxlen=512)

### Pre-Trained GloVe (Global Vectors for Word Representation) Embedding

In [23]:
model = build_model(embedding_matrix)
model.summary()

checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor='val_acc', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)

history = model.fit(
    x_train,
    y_train,
    batch_size=128,
    callbacks=[checkpoint],
    epochs=15,
    validation_split=0.3
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    6944700     ['input_1[0][0]']                
                                                                                                  
 spatial_dropout1d (SpatialDrop  (None, None, 300)   0           ['embedding[0][0]']              
 out1D)                                                                                           
                                                                                                  
 bidirectional (Bidirectional)  (None, None, 512)    1140736     ['spatial_dropout1d[0][0]']  

In [24]:
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt

y_pred = model.predict(x_test)
y_pred_argmax = np.argmax(y_pred, axis=1) + 1
y_test_argmax = np.argmax(y_test.values, axis=1) + 1

# print(cm)
print(classification_report(y_test_argmax, y_pred_argmax))

71/71 [==============================] - 124s 2s/step
              precision    recall  f1-score   support

           1       0.69      0.67      0.68       419
           2       0.15      0.11      0.13       136
           3       0.19      0.24      0.21       148
           4       0.28      0.17      0.21       309
           5       0.77      0.85      0.81      1241

    accuracy                           0.64      2253
   macro avg       0.42      0.41      0.41      2253
weighted avg       0.61      0.64      0.62      2253



In [25]:
statements = ["This product is great!", "I don't really like the product","this product is shit","irritated my eyes and didn't work","this product is okay, nothing special", "pretty okay product, not much to complain about"]

# Convert the input statements to numerical data using the same tokenizer that was used during training
sequences = tokenizer.texts_to_sequences(statements)
X = pad_sequences(sequences, maxlen=100)

# Make predictions using the trained model
predictions = model.predict(X)

# Print the predictions
for i, statement in enumerate(statements):
    listOfSentiments = predictions[i].tolist()
    # print(listOfSentiments)
    rating = listOfSentiments.index(max(listOfSentiments)) + 1
    print(f"Statement: {statement} | Sentiment: {rating} | Probability: {max(listOfSentiments)}")

1/1 [==============================] - 1s 912ms/step
Statement: This product is great! | Sentiment: 5 | Probability: 0.9536995887756348
Statement: I don't really like the product | Sentiment: 1 | Probability: 0.5354560017585754
Statement: this product is shit | Sentiment: 1 | Probability: 0.4589431583881378
Statement: irritated my eyes and didn't work | Sentiment: 1 | Probability: 0.8623155355453491
Statement: this product is okay, nothing special | Sentiment: 3 | Probability: 0.4560486376285553
Statement: pretty okay product, not much to complain about | Sentiment: 3 | Probability: 0.8185773491859436
